In [ ]:
from __future__ import print_function
from torch import nn, optim
import numpy as np
from random import randint
import copy                                                # to keep track of the evolutoin of the weights of the network
import os   
# from sklearn import manifold                               # for the t-SNE
# import matplotlib                                          # for the t-SNE
# from itertools import cycle, islice                        # for the t-SNE
from matplotlib import pyplot as plt 
from collections import OrderedDict
import pandas as pd
import datetime
import time
import torch                        
import torch.utils.data
from torch.nn import functional as F
# from torchvision import transforms
from datasets import YeastDataset # dataloader loading the yeast dataset in the correct format, replace it with one suitable for your problem
from data_elaboration_utilities import *
print("Libraries loaded")

In [ ]:
# define the arrays, the dictionary and the variables
repetitions = 50                                                              # number of random attempts

parameters = {"batch_size" : None, 
              "epochs" : None,                          
              "learning_rate" : None,
              "dataset_path" : None,
              "training_path" : None,
              "test_path" : None,
              # "num_workers" : 1, 
              # "pin_memory" : True,      
              "num_neurons" : None,
              "z_size" : None,
              "optimizer" : None,
              "dropout" : None,
              "reconstruction_loss" : None,        # reconstruction losses to use
              "reconstruction_weight" : None,    # weight of the reconstruction loss   # default  input_size
              "KLD_weight" : None,             # weight of the KLD loss      # default beta*z_size
              "beta" : None, 
              "gamma" : 0.1,
              "validation_loss" : None,
              "training_loss" : None,
              "KLD_loss" : None,                    # validation
              "reconstruction_loss_evol" : None,    # value of the reconstruction loss (validation)
              "experiment": None}

# batch_size = [32, 64, 128]            # same for training validation and test set, values for first part of the training

batch_size = {"fluxes" : [32, 64, 128], "genes" : [32, 64, 128], "all" : [32, 64, 128], 
              "RFECV_genes" : [32, 64, 128], "RFECV_fluxes" : [32, 64, 128], "RFECV_all" : [32, 64, 128],  
              "FCBF_fluxes" : [32, 64, 128], "FCBF_genes" : [32, 64, 128], "FCBF_all" : [32, 64, 128]}

# epochs = [400, 800, 1200, 1600, 2000, 2400]      # values for first part of the training

epochs = {"fluxes" : [400, 800, 1200, 1600, 2000, 2400], "genes" : [400, 800, 1200, 1600, 2000, 2400], 
          "all" : [400, 800, 1200, 1600, 2000, 2400], "RFECV_genes" : [400, 800, 1200, 1600, 2000, 2400], 
          "RFECV_fluxes" : [400, 800, 1200, 1600, 2000, 2400], "RFECV_all" : [400, 800, 1200, 1600, 2000, 2400], 
          "FCBF_fluxes" : [400, 800, 1200, 1600, 2000, 2400], "FCBF_genes" : [400, 800, 1200, 1600, 2000, 2400], 
          "FCBF_all" : [400, 800, 1200, 1600, 2000, 2400]}

# learning_rate =  [1e-2, 1e-3, 1e-4, 1e-5]        # values for first part of the training

learning_rate = {"fluxes" : [1e-2, 1e-3, 1e-4, 1e-5], "genes" : [1e-2, 1e-3, 1e-4, 1e-5], "all" : [1e-2, 1e-3, 1e-4, 1e-5], 
                 "RFECV_genes" : [1e-2, 1e-3, 1e-4, 1e-5], "RFECV_fluxes" : [1e-2, 1e-3, 1e-4, 1e-5], 
                 "RFECV_all" : [1e-2, 1e-3, 1e-4, 1e-5], "FCBF_fluxes" : [1e-2, 1e-3, 1e-4, 1e-5], 
                 "FCBF_genes" : [1e-2, 1e-3, 1e-4, 1e-5], "FCBF_all" : [1e-2, 1e-3, 1e-4, 1e-5]}

# dropout = [0, 0.3, 0.6]                          # values for first part of the training

dropout = {"fluxes" : [0, 0.3, 0.6], "genes" : [0, 0.3, 0.6], "all" : [0, 0.3, 0.6], 
           "RFECV_genes" : [0, 0.3, 0.6], "RFECV_fluxes" : [0, 0.3, 0.6], "RFECV_all" : [0, 0.3, 0.6], 
           "FCBF_fluxes" : [0, 0.3, 0.6], "FCBF_genes" : [0, 0.3, 0.6], "FCBF_all" : [0, 0.3, 0.6]}

beta = {"fluxes" : [3, 4, 5], "genes" : [3, 4, 5], "all" : [3, 4, 5], "RFECV_fluxes" : [3, 4, 5], 
        "RFECV_genes" : [3, 4, 5], "RFECV_all" : [3, 4, 5], "FCBF_fluxes" : [3, 4, 5], "FCBF_genes" : [3, 4, 5], 
        "FCBF_all" : [3, 4, 5]}

# optimizer = ["Adam", "SGD","Rprop", "Adadelta"]  # values for first part of the training

optimizer = {"fluxes" : ["Adam", "SGD","Rprop", "Adadelta"], "genes" : ["Adam", "SGD","Rprop", "Adadelta"], 
             "all" : ["Adam", "SGD","Rprop", "Adadelta"], "RFECV_genes" : ["Adam", "SGD","Rprop", "Adadelta"], 
             "RFECV_fluxes" : ["Adam", "SGD","Rprop", "Adadelta"], "RFECV_all" : ["Adam", "SGD","Rprop", "Adadelta"], 
             "FCBF_fluxes" : ["Adam", "SGD","Rprop", "Adadelta"], "FCBF_genes" : ["Adam", "SGD","Rprop", "Adadelta"], 
             "FCBF_all" : ["Adam", "SGD","Rprop", "Adadelta"]}

# loss = ["L1_loss", "MSE_loss", "Smooth_l1_loss"]     # values for first part of the training

reconstruction_loss = {"fluxes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], "genes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"],
                       "all" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], "RFECV_fluxes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"],
                       "RFECV_genes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], "RFECV_all" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], 
                       "FCBF_fluxes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"], "FCBF_genes" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"],
                       "FCBF_all" : ["L1_loss", "MSE_loss", "Smooth_l1_loss"]}


paths = ["path", None       # to distinguish between the two possible situations in which there is one or two datasets (not training, validation and test together)
        ]   # add the paths to the csv containing your data according to the usage of the variable paths below


no_cuda = False               # SHOULD BE FALSE                    
seed = 1                   
log_interval = 10       
percent_train = 0.7  
percent_validation = 0.2

dataset_path = paths[3] 
parameters["dataset_path"] = dataset_path

training_path = paths[-1]                                                      
test_path = paths[-1] 
parameters["training_path"] = training_path
parameters["test_path"] = test_path
                                               


input_size = 

reconstruction_weight = [input_size]          # default

# change the number of neurons to search according to the same of your data
# the keys of the dictionary correspond to different types of input data
num_neurons = {"fluxes" : [459, 520, 330, 200], "genes" : [6170, 6800, 3500, 2500], "all" : [6629, 7200, 4000, 2600], 
               "RFECV_genes" : [1858, 2350, 750, 550], "RFECV_all" : [1664, 2300, 600, 440], "RFECV_fluxes" : [19, 25, 7, 5], 
               "FCBF_genes" : [79, 90, 55, 40], "FCBF_fluxes" : [12, 18, 6, 4],"FCBF_all" : [68, 80, 46, 32]}  

# same as above
z_size = {"fluxes" : [200, 130, 80, 40], "genes" : [2500, 1200, 800, 500], "all" : [2600, 1300, 900, 600], 
          "RFECV_genes" : [550, 400, 320, 200], "RFECV_all" : [440, 280, 200, 170], "RFECV_fluxes" : [5, 4, 3, 2],
          "FCBF_genes" : [40, 32, 24, 15], "FCBF_fluxes" : [4, 3, 2, 1], "FCBF_all" : [32, 20, 16, 8]} 

# defines the mapping according to the input size
mapping = {459 : "fluxes", 6170 : "genes", 6629 : "all", 1858 : "RFECV_genes", 19 : "RFECV_fluxes", 1664 : "RFECV_all", 
           79 : "FCBF_genes", 12 : "FCBF_fluxes", 68 : "FCBF_all"}

num_n = mapping[input_size]

cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)

device = torch.device("cuda" if cuda else "cpu")              # launch on the GPU if available

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

if dataset_path != None:
    partition, labels = splitTrainingValidationTestSet(dataset_path, percent_train, percent_validation)
else:
    partition, labels = loadTrainingValidationTestSet(training_path, test_path, percent_train, percent_validation)
    
training_set = YeastDataset(partition['training'], labels)
validation_set = YeastDataset(partition['validation'], labels)
        
torch.set_printoptions(precision=9)                                 # to print more digits for the loss

print("Device: ", device)                           # simple check

In [ ]:
print(dataset_path)

In [5]:
def RandomIndexes():                                     # randomly choose the parameter
    int1 = randint(0, len(batch_size[num_n]) - 1)    
    int2 = randint(0, len(epochs[num_n]) - 1)
    int3 = randint(0, len(learning_rate[num_n]) - 1)
    int4 = randint(0, len(num_neurons[num_n]) - 1)                                  
    int5 = randint(0, len(z_size[num_n]) - 1)
    int6 = randint(0, len(optimizer[num_n]) - 1)
    int7 = randint(0, len(dropout[num_n]) - 1)
    int8 = randint(0, len(reconstruction_loss[num_n]) - 1) 
    int9 = randint(0, len(beta[num_n]) - 1) 
    return int1, int2, int3, int4, int5, int6, int7, int8, int9

def fillDict(d):
    ind1, ind2, ind3, ind4, ind5, ind6, ind7, ind8, ind9 = RandomIndexes()
    d["batch_size"] = batch_size[num_n][ind1]
    d["epochs"] = epochs[num_n][ind2]
    d["learning_rate"] = learning_rate[num_n][ind3]
    d["num_neurons"] = num_neurons[num_n][ind4]                          
    d["z_size"] = z_size[num_n][ind5]
    d["optimizer"] = optimizer[num_n][ind6]
    d["dropout"] = dropout[num_n][ind7]
    d["reconstruction_loss"] = reconstruction_loss[num_n][ind8]
    d["beta"] = beta[num_n][ind9]
    d = OrderedDict(sorted(d.items(), key=lambda t: t[0]))
    return d                                                                       

def checkIdentical(path, parameters):                        
    combinations = pd.read_csv(path) 
    if "Unnamed: 0" in combinations.columns:
        combinations.drop("Unnamed: 0", axis=1, inplace=True)        # change according to the structure of your file
    combinations = combinations.to_dict('records')
    if parameters in combinations:
        print("This combination of parameters has already been used")
        return (False, None)                                                        # return false if combination already used
    print(parameters)
    return (True, parameters)                                                       # true if combination not used yet

def checkForWarmStarting(path, num1, num2):                                     # returns False if no warmstarting is possible
    df = pd.read_csv(path)
    array = [num2]
    df = df.loc[(df['num_neurons'] == num1) & df['z_size'].isin(array)]
    if len(df) != 0:
        min_val = sorted(df["validation_loss"])                                # returns the one with the best validation loss
        exp = df.loc[df['validation_loss'] == min_val[0]]["experiment"]
        return exp.iloc[0]
    return False  

In [6]:
class trialVAE(nn.Module):
    def __init__(self, num_neurons, z_size, dropout):
        super(trialVAE, self).__init__()
    
        self.input_size = input_size
        self.num_neurons = num_neurons
        self.z_size = z_size
        self.dropout = dropout
        
        self.fc1 = nn.Linear(self.input_size, self.num_neurons)
        self.fc21 = nn.Linear(self.num_neurons, self.z_size)
        self.fc22 = nn.Linear(self.num_neurons, self.z_size)
        self.fc3 = nn.Linear(self.z_size, self.num_neurons)
        self.fc4 = nn.Linear(self.num_neurons, self.input_size)
        self.dropt = nn.Dropout(self.dropout)

    def encode(self, x):
        h1 = F.relu(self.fc1(x.float()))              
        # h1 = torch.tanh(self.fc1(x.float()))    # MSE doesn't learn and all the weights change at approximately each epoch
        return self.fc21(self.dropt(h1)), self.fc22(self.dropt(h1))

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        # h3 = torch.tanh(self.fc3(z))               # MSE doesn't learn and all the weights change at approximately each epoch
        return torch.sigmoid(self.fc4(self.dropt(h3)))         
        # return F.softmax(self.fc4(h3), dim=1)      # MSE doesn't decrease with it
        # return torch.tanh(self.fc4(h3))            # MSE doesn't decrease like with the sigmoid

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)        # representation to get
        return self.decode(z), mu, logvar

In [7]:
def loss_function(real, predict, loss, mu, logvar, epoch, tot_epoch):
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    if loss == "L1_loss":
        return torch.mean(reconstruction_weight[0]*F.l1_loss(predict, real) + KLD_weight*KLD), reconstruction_weight[0]*F.l1_loss(predict, real), KLD_weight*KLD
    elif loss == "MSE_loss":
        return torch.mean(reconstruction_weight[0]*F.mse_loss(predict, real) + KLD_weight*KLD), reconstruction_weight[0]*F.mse_loss(predict, real), KLD_weight*KLD
    else:
        return torch.mean(reconstruction_weight[0]*F.smooth_l1_loss(predict, real) + KLD_weight*KLD), reconstruction_weight[0]*F.smooth_l1_loss(predict, real), KLD_weight*KLD

In [8]:
def optimizer_function(optimizer, model, lr):
    if optimizer == "Adam":
        return optim.Adam(model.parameters(), lr=lr)
    elif optimizer == "SGD":
        return optim.SGD(model.parameters(), lr=lr, weight_decay=0.1)
    elif optimizer == "RMSprop":
        return optim.RMSprop(model.parameters(), lr=lr)
    else:
        return optim.Adadelta(model.parameters(), lr=lr)    

In [9]:
def train(epoch, lo, tot_epoch):   
    model.train()
    train_loss = 0
    for _, (data, labels) in enumerate(train_loader):
        # print("Epoch: {}, Batch index: {}".format(epoch, batch_idx))
        data = data.to(device)
        optmz.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss, reconstruction_loss, KLD = loss_function(data.float(), recon_batch, lo, mu, logvar, epoch, tot_epoch)
        # assert((reconstruction_loss == reconstruction_loss).item() == 1)  # check whether we're going to get NaNs
        loss.backward()
        train_loss += loss.item()
        optmz.step()
    with torch.no_grad():
        loss_evolution.append(loss.cpu().detach().numpy())   
        KLD_evolution.append(KLD.cpu().detach().numpy()) 
        reconstruction_loss_evolution.append(reconstruction_loss.cpu().detach().numpy()) 
        
    print('====> Epoch: {} Average loss: {:.4f}'.format(          
        epoch, loss / len(train_loader.dataset)))

    # print('====> Epoch: {} Average loss: {:.4f}'.format(
    #      epoch, train_loss / len(train_loader.dataset)))


def validate(epoch, lo, tot_epoch):
    model.eval()
    valid_loss = 0
    with torch.no_grad():
        for batch_idx, (data, labels) in enumerate(validation_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            loss, reconstruction_loss, KLD = loss_function(data.float(), recon_batch, lo, mu, logvar, epoch, tot_epoch)
            valid_loss += loss.item()
        valid_loss_evolution.append(loss.cpu().numpy()) 
        valid_KLD.append(KLD.cpu().numpy()) 
        valid_reconstruction.append(reconstruction_loss.cpu().numpy()) 
    # valid_loss /= len(validation_loader.dataset)                              
    # print('====> Validation set loss: {:.4f}'.format(valid_loss))

In [ ]:
for i in range(repetitions):
    while(True):
        flag, parameters = checkIdentical("path of file containing all the previously used combinations\Records.csv", fillDict(parameters))
        if flag:                                                              # if combination never used exit
            break
    directory_path = "path to the directory where to save the results" + "\\" + str(datetime.datetime.now().
                                                                                     strftime("%d_%m_%Y-%H.%M"))
    
    
    print("Repetition: ", i + 1)
    model = trialVAE(parameters["num_neurons"], parameters["z_size"], parameters["dropout"])
                                        # check for a possible warm starting
    check = checkForWarmStarting("path", # to the file containing all the tried combinations, different from the one above in case you want
                                 parameters["num_neurons"], parameters["z_size"])# to initialise the network with pre-computed weights
    if check :
        print("Warm starting from", check)
        model.load_state_dict(torch.load("path to the associated weights"), 
                                         strict=True)
    
    model.to(device)
    
    optmz = optimizer_function(parameters["optimizer"], model, parameters["learning_rate"])
    step_size = parameters["epochs"]//2
    gamma = parameters["gamma"]
    scheduler = optim.lr_scheduler.StepLR(optmz, step_size=step_size, gamma=gamma)
    
    KLD_weight = parameters["beta"]*parameters["z_size"]          # default
    
    train_loader = torch.utils.data.DataLoader(training_set, shuffle=True, batch_size=parameters["batch_size"], **kwargs)
    validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=parameters["batch_size"], shuffle=True, **kwargs)
    
    loss_evolution = list()                                                 # training losses
    KLD_evolution = list()
    reconstruction_loss_evolution = list()
    
    valid_loss_evolution = list()                                           # validation losses
    valid_KLD = list()
    valid_reconstruction = list()
    
    # training
    for epoch in range(1, parameters["epochs"] + 1):                                   
        scheduler.step()
        train(epoch, parameters["reconstruction_loss"], parameters["epochs"])
        validate(epoch, parameters["reconstruction_loss"], parameters["epochs"])  
        
    # save the model    
    if not os.path.exists(directory_path):                
        os.makedirs(directory_path)
    
    #save the losses for comparison (superfluous given the saves in the csv file)
    text_file = open("path\Losses.txt", "a")
    text_file.write("\n" + directory_path[68:] + "  " + str(valid_loss_evolution[-10:])) # change according to your directory_path
    text_file.close()
    
    torch.save(model.state_dict(), directory_path + "\\weights.pt")   
    
    visualizeLossesOverEpochs(valid_loss_evolution, loss_evolution, 0,       # comparison between validation and training losses
                          "Validation loss", "Training loss", "0", 50, 100, 'o-', False)
    
    plt.savefig(fname=directory_path + "\\validation_loss.png", bbox_inches="tight")
    
    visualizeLossesOverEpochs(loss_evolution, KLD_evolution, reconstruction_loss_evolution,
                          "Training loss", "KLD loss", "Reconstruction loss", 50, 100, 'o-', False)  # comparison between training, KLD and reconstruction losses

    plt.savefig(fname=directory_path + "\\training_loss.png", bbox_inches="tight")
    
    # save full validation loss
    text_file = open(directory_path +"\\validation_loss.txt", "w")
    text_file.write(str(valid_loss_evolution) + "\n")
    text_file.close()
    
    # save full KLD loss (validation)
    text_file = open(directory_path +"\\KLD_loss.txt", "w")
    text_file.write(str(valid_KLD) + "\n")
    text_file.close()
    
    # save full reconstruction loss (validation)
    text_file = open(directory_path +"\\reconstruction_loss.txt", "w")
    text_file.write(str(valid_reconstruction) + "\n")
    text_file.close()
    
    # write the parameters in the csv file
    combinations = pd.read_csv("path of file containing all the previously used combinations\Records.csv") 
    if "Unnamed: 0" in combinations.columns:
        combinations.drop("Unnamed: 0", axis=1, inplace=True)
    combinations = combinations.to_dict('records')
    parameters["training_loss"] = loss_evolution[-1]
    parameters["validation_loss"] = valid_loss_evolution[-1]
    parameters["KLD_loss"] = valid_KLD[-1]                                  # N.B. these are the validation losses
    parameters["KLD_weight"] = KLD_weight
    parameters["reconstruction_weight"] = reconstruction_weight[0]
    parameters["reconstruction_loss_evol"] = valid_reconstruction[-1]
    parameters["training_path"] = str(training_path)
    parameters["test_path"] = str(test_path)
    parameters["experiment"] = directory_path[68:]
    combinations.append(parameters)                                               
    records = pd.DataFrame(combinations)
    records.to_csv("path of file containing all the previously used combinations\Records.csv", encoding="utf-8")
    
    text_file = open(directory_path + "\\" + "Parameters.txt", "w")
    text_file.write("Parameters used: \n\n")
    for _, (key, value) in enumerate(parameters.items()):
        text_file.write(key + " = " + str(value) + "\n")
    text_file.write("\n Dataset path: " + dataset_path)
    text_file.write("\n Training set path: " + str(training_path))  
    text_file.write("\n Test set path: " + str(test_path))
    
    text_file.close()
    
    plt.close("all")                                            # close all the figures in order to save memory